In [2]:
# imports, add to this as needed

# change to matplotlib notebook for classic view
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
from scipy.optimize import minimize
import json
from simanneal import Annealer

***Note:  this is a new version of HW5 that we wrote in Summer 2020 (added a few hints and fixes on 10/7/20).  Please fire away with questions on Piazza as we know the first draft of anything often needs clarification!***

# **HW5.1** - The Traveling Salesman Problem

In Lesson 4 we saw how to use local search to seek solutions to the TSP problem with 48 cities.  In this homework problem you'll use the `simanneal` package and the genetic algorithm to search for solutions.  The json file Caps48.json in the data folder has both the distance matrix and the optimal tour.  Your goal is to identify tours with total distance under 19,000 kilometers with each algorithm.

The following cell shows you how to visualize a tour of the 48 cities (and reminds you how to use the json file).  You should plot your solutions to see if they seem reasonable.  A good solution will avoid long detours and shouldn't intersect itself.  We plot the best possible solution here:

In [3]:
with open("data/Caps48.json", "r") as tsp_data:
    tsp = json.load(tsp_data)
distance_matrix = tsp["DistanceMatrix"]
optimal_tour = tsp["OptTour"]
opt_dist = tsp["OptDistance"]/1000 # converted to kilometers
xy = np.array(tsp["Coordinates"])

def plot_tour(best_tour, xy_meters, best_dist, height, width):

    meters_to_pxl = 0.0004374627441064968
    intercept_x = 2.464
    intercept_y = 1342.546
    xy_pixels = np.zeros(xy_meters.shape)
    xy_pixels[:,0] = meters_to_pxl * xy_meters[:,0] + intercept_x
    xy_pixels[:,1] = -meters_to_pxl * xy_meters[:,1] + intercept_y

    fig, ax = plt.subplots(1, 1, figsize=(height, width))
    im = plt.imread('images/caps48.png')
    implot = ax.imshow(im)
    plt.setp(ax.get_xticklabels(), visible=False)
    plt.setp(ax.get_yticklabels(), visible=False)
    ax.tick_params(axis='both', which='both', length=0)

    loop_tour = np.append(best_tour, best_tour[0])
    ax.plot(xy_pixels[loop_tour, 0],
            xy_pixels[loop_tour, 1],
            c='b',
            linewidth=1,
            linestyle='-')
    plt.title('Best Distance {:d} km'.format(int(best_dist)))

# this is an example of how to plot a tour
plot_tour(optimal_tour, xy, opt_dist, 9, 6)

**HW5.1a** -

There isn't much to this part.  Adapt the code in the lesson to use the `simanneal` package to find a tour with total distance less than 19,000 km.  Plot your tour.  You may have to experiment with the temperature schedule to achieve a good result.  Without hacking the `simanneal` code it's impossible to set a random number seed to make reproducible results so just give this a good try.  You can tell us about your best solution even if it isn't the latest solution you found.  A plot isn't required, but it is useful for inspecting the tour to see if it is at least sensible.

<font color = "blue"> *** 4 points -  answer in cell below *** (don't delete this cell) </font>

**HW5.1b** -

Use the Genetic Algorithm described in the text to find a tour with total distance less than 19,000 km.

* Initialize the population with permuations
* Implement the Ordered Crossover operator described in the lesson.
* Implement the Shuffling Indices (random city swaps) described in the lesson.
* Experiment with the algorithm parameters to find a good tour.  Add a line at the top to make the results reproducible: `np.random.seed(123)`.  You can change the seed to get different results, but the same seed with the same parameters should produce the same tour.
* Feel free to discuss parameter choices with other students on Piazza.
* Include a plot of your tour.
* To get you started we've copied the GA code from the lesson and added ###### before and after places you'll need to make changes.

<font color = "blue"> *** 10 points -  answer in cell below *** (don't delete this cell) </font>

In [0]:
######
pop_size = 100 # should be even due to the way we'll implement crossover
ind_size = 10 # determines number of input variables for Rastrigin and each individual
######
#lower = -5.12 # lower and upper bounds on the real variables
#upper = 5.12
tourn_size = 3 # tournament size for selection
cx_prob = 0.8 # probability a pair of parents crossover to produce two children
mut_prob = 0.2 # probability an individual mutates
ind_prob = 0.1 # probability each variable in an individual mutates
#sigma = (upper-lower)/6 # standard deviation (scale) for gaussian mutations
###### maybe here
num_iter = 2000 # number of genetic algorithm mutations
update_iter = 100 # how often to display output
######

stats = np.zeros((num_iter+1,3)) # for collecting statistics

######
# objective or fitness function
def rastrigin(x):
    x = np.array(x) # force a numpy arrray here so that the math below works
    return np.sum(x**2 + 10 - 10 * np.cos(2 * np.pi * x) )
######

###### may need to also enforce that the variables are integers and not floats
#initialize population and fitness np.random.randint(0,2,size=20,dtype=bool)
pop = np.random.uniform(low=lower, high=upper, size = (ind_size,pop_size))
fitness = np.zeros(pop_size)
for j in range(pop_size):
    fitness[j] = rastrigin(pop[:,j])
######

# initialize stats and output
best_fitness = min(fitness)
stats[0,:] = np.array([0,best_fitness, best_fitness])
print('Iteration | Best this iter |    Best ever')

for iter in range(num_iter):
    # tournament selection
    sorted_pos = fitness.argsort() # sort pop by increasing fitness
    fitness = fitness[sorted_pos]
    pop = pop[:,sorted_pos]
    select_pop = np.zeros((ind_size,pop_size)) # initialize selected population
    for j in range(pop_size):
        subset_pos = np.random.choice(pop_size,tourn_size,replace=False) # select without replacement
        smallest_pos = np.min(subset_pos) # choose index corresponding to lowest fitness
        select_pop[:,j] = pop[:,smallest_pos]

    # one-point crossover (mating)
    cx_pop = np.zeros((ind_size,pop_size)) # initialize crossover population
    for j in range(int(pop_size/2)):  # pop_size must be even
        #######
        parent1, parent2 = select_pop[:,2*j], select_pop[:,2*j+1]
        child1, child2 = parent1.copy(), parent2.copy()
        if np.random.uniform() < cx_prob: # crossover occurs
            cx_point = np.random.randint(1,ind_size) # crossover point between 0 and ind_size-2
            child1[0:cx_point], child2[0:cx_point] = parent2[0:cx_point], parent1[0:cx_point]
        cx_pop[:,2*j] = child1
        cx_pop[:,2*j+1] = child2
        ######

    # gaussian mutation (rewritten to remove nested loop for speed)
    mut_pop = np.zeros((ind_size,pop_size)) # initialize mutation population
    for j in range(pop_size):
        individual = cx_pop[:,j].copy() # copy is necessary to avoid conflicts in memory
        if np.random.uniform()<mut_prob:
            ###### swap the ith entry with a randomly selected entry with prob ind_prob
            individual = individual + np.random.normal(0,sigma,ind_size)*(np.random.uniform(size=ind_size)<ind_prob)
            individual = np.maximum(individual,lower) # clip to lower bound
            individual = np.minimum(individual,upper) # clip to upper bound
            ######
        mut_pop[:,j] = individual.copy() # copy is necessary to avoid conflicts in memory

    # fitness evaluation with local search
    pop = mut_pop.copy()
    for j in range(pop_size):
        fitness[j] = rastrigin(pop[:,j])

    # collect stats and output to screen
    min_fitness = min(fitness) # best for this iteration
    if min_fitness < best_fitness: # best for all iterations
        best_fitness = min_fitness
        index = np.argmin(fitness)
        best_x = pop[:,index]

    stats[iter+1,:] = np.array([iter+1,min_fitness, best_fitness])
    if (iter+1) % update_iter == 0:
        print(f"{stats[iter+1,0]:9.0f} | {stats[iter+1,1]:14.3e} | {stats[iter+1,2]:12.3e}")

######
print(f"The minimum value found of the Rastrigin function is {best_fitness:.4f}")
print("The location of that minimum is:")
######
print('(',', '.join(f"{x:.4f}" for x in best_x),')')

# **HW5.2** - The Knapsack Problem

We first encountered this problem in the Lesson 4 Homework.

The knapsack problem is a classical combinatorial optimization problem that will be good for practicing with the ideas of discrete local search and multistart.  Given a set of items, each with a weight and a value, determine which items to include in a collection so that the total weight is less than or equal to a given limit and the total value is as large as possible.  In the 0-1 version of the knapsack problem, the decision variables are binary (or boolean) and represent whether or not to include each item in the collection.  We'll start with 20 items and you need to determine the collection of items that maximizes the value and keeps the total weight no more than 50 (that is $\leq 50$).

In [9]:
# generate random weights and values for a knapsack problem
# DO NOT CHANGE ANYTHING in this block of code
import numpy as np
num_items = 20
np.random.seed(seed=123) 
values = np.random.randint(low=5, high=50, size=num_items)
weights = np.random.randint(low=1, high=10, size=num_items)
max_weight = 50
np.random.seed() # use system clock to reset the seed so future random numbers will appear random

Use the `simanneal` package to apply simulated annealing to finding a good solution to this knapsack problem.  Remember that the knapsack problem is a maximization problem so you'll have to work with the appropriate negate

For this problem we have the constraint that total weight $\leq 50$ and we'll take two different approaches to ensuring our solutions satisfy the constraint.  (The two constraint approaches are also discussed in the supplemental notebooks in the extras folder.  Note that we are no longer using the DEAP package for the genetic algorithm that is mentioned in the supplements.)

**NOTE:**  The optimal backpack has total value 435 and total weight 50.  All of the algorithms below should be able to easily find this solution.

**HW5.2a** - The first approach is called a **hard constraint** wherein we reject all possible solutions that don't satisfy the constraint.  You'll implement this in the move() method.  
* copy `self.state` with `new_state = self.state.copy()`
* make a move on `new_state` (toggle a random item like in HW 4)
* if the total weight of `new_state` is $\leq 50$ then `set self.state = new_state`
* else don't change `self.state`.

Find a "good" solution to the knapsack problem by using `simanneal` with a hard constraint.

<font color = "blue"> *** 5 points -  answer in cell below *** (don't delete this cell) </font>

**HW5.2b** - The second approach is called a **soft constraint** because its possible to find a solution that doesn't satisfy the constraint.  Soft constraints are implemented by including a penalty in the objective function when the proposed solution doesn't satisfy the constraint.  To do this you'll modify the function called by the energy() method:
```
def knapsack_value_penalty(x, values, weights, max_tot_weight):
    # x is a vector of booleans of which items to include
    tot_value = sum(values[x])
    penalty = sum(values)*min( max_tot_weight - sum(weights[x]), 0) 
    return tot_value+penalty
```
The penalty here is negative when the total weight is too large so that the optimizer knows it hasn't found a good maximizing solution.

Find a "good" solution to the knapsack problem by using simanneal with a soft constraint.

<font color = "blue"> *** 5 points -  answer in cell below *** (don't delete this cell) </font>

**HW5.2c** - Use the soft constraint approach to solve the knapsack problem with the genetic algorithm (use the negated, penalized objective function).  Use OnePoint Crossover for mating.  Mutation should be toggling (flipping) each variable with probability `ind_prob`. Use tournament selection.  Include your solution in the cell below.

<font color = "blue"> *** 6 points -  answer in cell below *** (don't delete this cell) </font>

**HW5.2d** - Which of the three algorithms above seems to work the best.  Do you think you've found the knapsack with highest possible value (the global max)?  Why or why not?

<font color = "blue"> *** 2 points -  answer in cell below *** (don't delete this cell) </font>

<font color = "green">
replace this text with answer, your answer should be green   
</font>

# **HW5.3** - The 10-dimensional Rastrigin Function with Local Search

The Rastrigin function was first explored in Lesson 4 and was also used in the Genetic Algorithm example in Lesson 5.  Here is the definition of the function:

In [6]:
# rastrigin definition
def rastrigin(x):
    x = np.array(x) # force a numpy arrray here so that the math below works
    # pass a single vector of length n (=dim) to evaluate Rastrigin
    return sum(x**2 + 10 - 10 * np.cos(2 * np.pi * x))

This function can be applied to an `x` that is any iterable of any length.  We'll be working with numpy arrays.  For the 10 dimensional problems your numpy array should be a vector with 10 floating point numbers.  We'll enforce bounds that $-5.12 \leq x_i \leq 5.12$ so all of the entries in $x$ should be between those bounds.  Your goal is find the global minimizer at (0,0,0,...) where the function value is also zero.

**HW5.3a** - Use the `simanneal` package.  The self-assessment solutions should be helpful here. This solution does not involve local search.

<font color = "blue"> *** 5 points -  answer in cell below *** (don't delete this cell) </font>

**HW5.3b** - Now include local search in your `simanneal` solution.  To do this you'll need to modify the move() method. After you've add the random change and clipped to bounds, then start a local search from the current self.state using scipy.optimize.minimize (include the bounds) and set self.state to the resulting location found by the local search.  You can tell the minimize function to optimize self.energy starting from self.state.  To speed this up you can make the local searches with low precision by adding `tol=0.1` to the minimize() function call.  At the very end you can do one final local search without the added `tol=0.1` to improve your final answer.

<font color = "blue"> *** 5 points -  answer in cell below *** (don't delete this cell) </font>

**HW5.3c** - Now include local search with the genetic algorithm approach developed in Lesson 5.

You're going to make two changes here.

* Replace one point crossover with blended crossover (described in the lesson) with $\alpha = 0.2$.  You should make sure the resulting children satisfy the bounds.  
* Add local search to the genetic algorithm.  At the top of the main loop, before tournament selection.

We've copied the code from the lesson and added comments on where to make the modifications.  To get your code working you should reduce the pop_size and iterations.  Experiment to get a good result.


<font color = "blue"> *** 6 points -  answer in cell below *** (don't delete this cell) </font>

In [3]:
pop_size = 100 # should be even due to the way we'll implement crossover
ind_size = 10 # determines number of input variables for Rastrigin and each individual
lower = -5.12 # lower and upper bounds on the real variables
upper = 5.12
tourn_size = 3 # tournament size for selection
cx_prob = 0.8 # probability a pair of parents crossover to produce two children
mut_prob = 0.2 # probability an individual mutates
ind_prob = 0.1 # probability each variable in an individual mutates
sigma = (upper-lower)/6 # standard deviation (scale) for gaussian mutations
num_iter = 2000 # number of genetic algorithm mutations
update_iter = 100 # how often to display output

stats = np.zeros((num_iter+1,3)) # for collecting statistics

# objective or fitness function
def rastrigin(x):
    x = np.array(x) # force a numpy arrray here so that the math below works
    return np.sum(x**2 + 10 - 10 * np.cos(2 * np.pi * x) )

#initialize population and fitness
pop = np.random.uniform(low=lower, high=upper, size = (ind_size,pop_size))
fitness = np.zeros(pop_size)
for j in range(pop_size):
    fitness[j] = rastrigin(pop[:,j])

# initialize stats and output
best_fitness = min(fitness)
stats[0,:] = np.array([0,best_fitness, best_fitness])
print('Iteration | Best this iter |    Best ever')

for iter in range(num_iter):
    ### CHANGE - local search goes here
    #    - sort pop by increasing fitness
    #    - take first three individuals with lowest fitness and replace them by the minimizing location resulting 
    #      from using scipy.optimize.minimize with bounds applied to each individual each individual
    
    # tournament selection
    sorted_pos = fitness.argsort() # sort pop by increasing fitness
    fitness = fitness[sorted_pos]
    pop = pop[:,sorted_pos]
    select_pop = np.zeros((ind_size,pop_size)) # initialize selected population
    for j in range(pop_size):
        subset_pos = np.random.choice(pop_size,tourn_size,replace=False) # select without replacement
        smallest_pos = np.min(subset_pos) # choose index corresponding to lowest fitness
        select_pop[:,j] = pop[:,smallest_pos]

    ### CHANGE this to blended crossover
    # one-point crossover (mating)
    cx_pop = np.zeros((ind_size,pop_size)) # initialize crossover population
    for j in range(int(pop_size/2)):  # pop_size must be even
        parent1, parent2 = select_pop[:,2*j], select_pop[:,2*j+1]
        child1, child2 = parent1.copy(), parent2.copy()
        if np.random.uniform() < cx_prob: # crossover occurs
            cx_point = np.random.randint(1,ind_size) # crossover point between 0 and ind_size-2
            child1[0:cx_point], child2[0:cx_point] = parent2[0:cx_point], parent1[0:cx_point]
        cx_pop[:,2*j] = child1
        cx_pop[:,2*j+1] = child2

    # gaussian mutation (rewritten to remove nested loop for speed)
    mut_pop = np.zeros((ind_size,pop_size)) # initialize mutation population
    for j in range(pop_size):
        individual = cx_pop[:,j].copy() # copy is necessary to avoid conflicts in memory
        if np.random.uniform()<mut_prob:
            individual = individual + np.random.normal(0,sigma,ind_size)*(np.random.uniform(size=ind_size)<ind_prob)
            individual = np.maximum(individual,lower) # clip to lower bound
            individual = np.minimum(individual,upper) # clip to upper bound
        mut_pop[:,j] = individual.copy() # copy is necessary to avoid conflicts in memory

    # fitness evaluation with local search
    pop = mut_pop.copy()
    for j in range(pop_size):
        fitness[j] = rastrigin(pop[:,j])

    # collect stats and output to screen
    min_fitness = min(fitness) # best for this iteration
    if min_fitness < best_fitness: # best for all iterations
        best_fitness = min_fitness
        index = np.argmin(fitness)
        best_x = pop[:,index]

    stats[iter+1,:] = np.array([iter+1,min_fitness, best_fitness])
    if (iter+1) % update_iter == 0:
        print(f"{stats[iter+1,0]:9.0f} | {stats[iter+1,1]:14.3e} | {stats[iter+1,2]:12.3e}")
        
print(f"The minimum value found of the Rastrigin function is {best_fitness:.4f}")
print("The location of that minimum is:")
print('(',', '.join(f"{x:.4f}" for x in best_x),')')

Iteration | Best this iter |    Best ever


      100 |      1.748e+00 |    1.748e+00


      200 |      1.468e+00 |    1.183e+00


      300 |      5.602e-01 |    5.602e-01


      400 |      4.824e-01 |    4.824e-01


...

     1600 |      1.953e-02 |    1.953e-02


     1700 |      1.953e-02 |    1.353e-02


     1800 |      1.435e-02 |    1.353e-02


     1900 |      8.399e-03 |    8.399e-03


     2000 |      5.694e-03 |    5.694e-03
The minimum value found of the Rastrigin function is 0.0057
The location of that minimum is:
( 0.0006, 0.0006, 0.0023, -0.0024, -0.0025, -0.0018, 0.0011, 0.0024, 0.0006, -0.0004 )


**HW5.3d** - 

Briefly summarize the results of the the three algorithms above.  Which seems to work best in terms of accuracy and in terms of efficiency?

<font color = "blue"> *** 2 points -  answer in cell below *** (don't delete this cell) </font>

<font color = "green">
replace this text with answer, your answer should be green   
</font>